In [45]:
import urllib2, os, sys, subprocess
import googleapiclient.discovery
import os, sys
import random
import time 

In [8]:
def read_instance_properties():
    #compute-nodes: list of other instances that belong to cluster. Try using series-[1-100] notation
    #name of slurm-master 
    #machine type, can we get that from /proc/cpuinfo? 
    pass

0

In [17]:
def num_cpus():
    try:
        np=subprocess.check_output('cat /proc/cpuinfo | grep processor | wc -l', shell=True)
        p = int(np)
        return p
    except:
        p = 1 
        return p 

In [28]:
def machine_type():
    #Cat proc/cpuinfo, and some memory ? 
    cpus = num_cpus()
    machine = {'sockets': 1, 'cores': cpus/2, 'threads': 2, 'memory': 1000}
    return machine 
#TODO: Convert to string type so that we can use it? 
    

In [61]:
slurm_master="ubslurm1"
compute_nodes="bravo[1-10]"
mc_string = "CPUs=1 sockets=1 cores=1 threads=1" 
machine=machine_type()

In [69]:
machine={'sockets': 1, 'cores': 1, 'threads': 2, 'memory': 1000}

In [70]:
slurmconfstr = """
# slurm.conf file generated by configurator easy.html.
# Put this file on all nodes of your cluster.
# See the slurm.conf man page for more information.
#
ControlMachine={slurm_master}
AuthType=auth/none
#CheckpointType=checkpoint/none
#CryptoType=crypto/munge
#ControlAddr=
#
#MailProg=/bin/mail
MpiDefault=none
#MpiParams=ports=#-#
ProctrackType=proctrack/pgid
ReturnToService=1
SlurmctldPidFile=/var/run/slurm-llnl/slurmctld.pid
#SlurmctldPort=6817
SlurmdPidFile=/var/run/slurm-llnl/slurmd.pid
#SlurmdPort=6818
SlurmdSpoolDir=/var/lib/slurm-llnl/slurmd
SlurmUser=slurm
#SlurmdUser=root
StateSaveLocation=/var/lib/slurm-llnl/slurmctld
SwitchType=switch/none
TaskPlugin=task/none
JobCredentialPrivateKey=/home/prateek3_14/slurmkey
#
#
# TIMERS
#KillWait=30
#MinJobAge=300
#SlurmctldTimeout=120
#SlurmdTimeout=300
#
#
# SCHEDULING
FastSchedule=1
SchedulerType=sched/builtin
#SchedulerPort=7321
SelectType=select/linear
#
#
# LOGGING AND ACCOUNTING
AccountingStorageType=accounting_storage/none
ClusterName=ubslurm1
#JobAcctGatherFrequency=30
JobAcctGatherType=jobacct_gather/none
#SlurmctldDebug=3
SlurmctldLogFile=/var/log/slurm-llnl/slurmctld.log
#SlurmdDebug=3
SlurmdLogFile=/var/log/slurm-llnl/slurmd.log
#
#
# COMPUTE NODES
""".format(slurm_master=slurm_master)

slurmconfstr += ' '.join(("NodeName=DEFAULT",
                      "Sockets="        + str(machine['sockets']),
                      "CoresPerSocket=" + str(machine['cores']),
                      "ThreadsPerCore=" + str(machine['threads']),
                      "State=UNKNOWN"))

slurmconfstr += '\n'
slurmconfstr += 'NodeName={slurm_master} \n'.format(slurm_master=slurm_master)
slurmconfstr += 'NodeName={compute_nodes} \n'.format(compute_nodes=compute_nodes)


slurmconfstr += "\n PartitionName=long Nodes={compute_nodes} Default=YES MaxTime=INFINITE State=UP \n".format(compute_nodes=compute_nodes)

In [71]:
print slurmconfstr


# slurm.conf file generated by configurator easy.html.
# Put this file on all nodes of your cluster.
# See the slurm.conf man page for more information.
#
ControlMachine=ubslurm1
AuthType=auth/none
#CheckpointType=checkpoint/none
#CryptoType=crypto/munge
#ControlAddr=
#
#MailProg=/bin/mail
MpiDefault=none
#MpiParams=ports=#-#
ProctrackType=proctrack/pgid
ReturnToService=1
SlurmctldPidFile=/var/run/slurm-llnl/slurmctld.pid
#SlurmctldPort=6817
SlurmdPidFile=/var/run/slurm-llnl/slurmd.pid
#SlurmdPort=6818
SlurmdSpoolDir=/var/lib/slurm-llnl/slurmd
SlurmUser=slurm
#SlurmdUser=root
StateSaveLocation=/var/lib/slurm-llnl/slurmctld
SwitchType=switch/none
TaskPlugin=task/none
JobCredentialPrivateKey=/home/prateek3_14/slurmkey
#
#
# TIMERS
#KillWait=30
#MinJobAge=300
#SlurmctldTimeout=120
#SlurmdTimeout=300
#
#
# SCHEDULING
FastSchedule=1
SchedulerType=sched/builtin
#SchedulerPort=7321
SelectType=select/linear
#
#
# LOGGING AND ACCOUNTING
AccountingStorageType=accounting_storage/none
ClusterName

In [72]:
startupscriptstr = """#!/bin/bash

logger "Startup Script Begins.... "
logger "Running as `whoami`" 

#systemctl stop slurmd 

cat <<\EOF >> /etc/slurm-llnl/slurm.conf
{slurmconfstr}
EOF

systemctl start slurmd 

logger "Slurm conf applied, startup script ending" 

exit 0

""".format(slurmconfstr=slurmconfstr)

In [73]:
print(startupscriptstr)

#!/bin/bash

logger "Startup Script Begins.... "
logger "Running as `whoami`" 

#systemctl stop slurmd 

cat <<\EOF >> /etc/slurm-llnl/slurm.conf

# slurm.conf file generated by configurator easy.html.
# Put this file on all nodes of your cluster.
# See the slurm.conf man page for more information.
#
ControlMachine=ubslurm1
AuthType=auth/none
#CheckpointType=checkpoint/none
#CryptoType=crypto/munge
#ControlAddr=
#
#MailProg=/bin/mail
MpiDefault=none
#MpiParams=ports=#-#
ProctrackType=proctrack/pgid
ReturnToService=1
SlurmctldPidFile=/var/run/slurm-llnl/slurmctld.pid
#SlurmctldPort=6817
SlurmdPidFile=/var/run/slurm-llnl/slurmd.pid
#SlurmdPort=6818
SlurmdSpoolDir=/var/lib/slurm-llnl/slurmd
SlurmUser=slurm
#SlurmdUser=root
StateSaveLocation=/var/lib/slurm-llnl/slurmctld
SwitchType=switch/none
TaskPlugin=task/none
JobCredentialPrivateKey=/home/prateek3_14/slurmkey
#
#
# TIMERS
#KillWait=30
#MinJobAge=300
#SlurmctldTimeout=120
#SlurmdTimeout=300
#
#
# SCHEDULING
FastSchedule=1
SchedulerType

In [88]:
project='first-220321'
ze='us-east1-b'
vmnum=0

In [83]:
compute = googleapiclient.discovery.build('compute', 'v1')

In [80]:
compute.instances().list(project=project,zone=ze).execute()

{u'id': u'projects/first-220321/zones/us-central1-f/instances',
 u'items': [{u'cpuPlatform': u'Unknown CPU Platform',
   u'creationTimestamp': u'2019-02-18T11:23:26.689-08:00',
   u'deletionProtection': False,
   u'disks': [{u'autoDelete': True,
     u'boot': True,
     u'deviceName': u'persistent-disk-0',
     u'guestOsFeatures': [{u'type': u'VIRTIO_SCSI_MULTIQUEUE'}],
     u'index': 0,
     u'interface': u'SCSI',
     u'kind': u'compute#attachedDisk',
     u'licenses': [u'https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/licenses/ubuntu-1804-lts'],
     u'mode': u'READ_WRITE',
     u'source': u'https://www.googleapis.com/compute/v1/projects/first-220321/zones/us-central1-f/disks/aardvark10',
     u'type': u'PERSISTENT'}],
   u'id': u'8839863289518667970',
   u'kind': u'compute#instance',
   u'labelFingerprint': u'42WmSpB8rSM=',
   u'machineType': u'https://www.googleapis.com/compute/v1/projects/first-220321/zones/us-central1-f/machineTypes/n1-standard-16',
   u'me

In [96]:
def start_instance(mtype, zone, name=None):
    global wordlist 
    global vmnum 
    
    vmnum = vmnum+1 
    
    if name is None:
        name = "bravo"+str(vmnum)
    
    machine_type = "zones/{}/machineTypes/{}".format(zone, mtype)
    
    
    instance_body={
        'name':name,
        'machineType':machine_type, 
        'scheduling':
          {
        'preemptible': 'true'
          },
        'disks': [
        {
            'boot': True,
            'autoDelete': True,
            'initializeParams': {
                'sourceImage': "global/images/ubs2"
                }
        }],
        'metadata' : {
            "items" : [
                {
                    "key": "startup-script",
                    "value":startupscriptstr
                }
            ]
        },
        'networkInterfaces': [{
        'network': 'global/networks/default',
        'accessConfigs': [
            {'type': 'ONE_TO_ONE_NAT', 'name': 'External NAT'}
        ]
    }],
    }
    print(str(instance_body))
    
    response = compute.instances().insert(project=project,zone=zone,body=instance_body).execute()
    return response 
    

In [100]:
mtype='n1-standard-2'
zone=ze 
name="bravo4"

In [101]:
d

{'val': '#!/bin/bash\n\nlogger "Startup Script Begins.... "\nlogger "Running as `whoami`" \n\n#systemctl stop slurmd \n\ncat <<\\EOF >> /etc/slurm-llnl/slurm.conf\n\n# slurm.conf file generated by configurator easy.html.\n# Put this file on all nodes of your cluster.\n# See the slurm.conf man page for more information.\n#\nControlMachine=slmaster\nAuthType=auth/none\n#CheckpointType=checkpoint/none\n#CryptoType=crypto/munge\n#ControlAddr=\n#\n#MailProg=/bin/mail\nMpiDefault=none\n#MpiParams=ports=#-#\nProctrackType=proctrack/pgid\nReturnToService=1\nSlurmctldPidFile=/var/run/slurm-llnl/slurmctld.pid\n#SlurmctldPort=6817\nSlurmdPidFile=/var/run/slurm-llnl/slurmd.pid\n#SlurmdPort=6818\nSlurmdSpoolDir=/var/lib/slurm-llnl/slurmd\nSlurmUser=slurm\n#SlurmdUser=root\nStateSaveLocation=/var/lib/slurm-llnl/slurmctld\nSwitchType=switch/none\nTaskPlugin=task/none\nJobCredentialPrivateKey=/home/prateek3_14/slurmkey\n#\n#\n# TIMERS\n#KillWait=30\n#MinJobAge=300\n#SlurmctldTimeout=120\n#SlurmdTimeou

In [102]:
response = start_instance(mtype,ze,name)

{'machineType': 'zones/us-east1-b/machineTypes/n1-standard-2', 'name': 'bravo4', 'disks': [{'initializeParams': {'sourceImage': 'global/images/ubs2'}, 'boot': True, 'autoDelete': True}], 'scheduling': {'preemptible': 'true'}, 'metadata': {'items': [{'value': '#!/bin/bash\n\nlogger "Startup Script Begins.... "\nlogger "Running as `whoami`" \n\n#systemctl stop slurmd \n\ncat <<\\EOF >> /etc/slurm-llnl/slurm.conf\n\n# slurm.conf file generated by configurator easy.html.\n# Put this file on all nodes of your cluster.\n# See the slurm.conf man page for more information.\n#\nControlMachine=ubslurm1\nAuthType=auth/none\n#CheckpointType=checkpoint/none\n#CryptoType=crypto/munge\n#ControlAddr=\n#\n#MailProg=/bin/mail\nMpiDefault=none\n#MpiParams=ports=#-#\nProctrackType=proctrack/pgid\nReturnToService=1\nSlurmctldPidFile=/var/run/slurm-llnl/slurmctld.pid\n#SlurmctldPort=6817\nSlurmdPidFile=/var/run/slurm-llnl/slurmd.pid\n#SlurmdPort=6818\nSlurmdSpoolDir=/var/lib/slurm-llnl/slurmd\nSlurmUser=slu